# DDR Final Project: Yelp Resturant Reviews 

Objective: Web scrape resturants on Yelp to identify locations for new resturant 

In [1]:
# load packages 
# import libraries
import mysql.connector
import warnings

import requests
import json
import codecs
from bs4 import BeautifulSoup


In [2]:
# define headers and url of the page
api_key = 'ovW_u_S0Cu9Csv9rfDH3ok2LeYO88IGfAiBvnkqblgRGPrwb0K8MOPsk99Ac6k3ESqxfUkRwE_xjk-GtjHkha5hAggJJ_3nZIU64F0GW8si_mkhFY2ByVTHu-hfoZXYx'
url = 'https://api.yelp.com/v3/businesses/search'

In [60]:
# define headers and parameters 
headers = {'Authorization': 'bearer %s' % api_key}
params = {'term': 'restaurants', 
          #'limit': 400,
          'radius': 20000,
          'price': 4,
          'location': 'San Francisco'}

# make a request to Yelp
page = requests.get(url,headers=headers, params=params)
response = BeautifulSoup(page.content, 'html.parser')

# put in json format 
restaurants = json.loads(str(response))

# look at the keys in the json dict 
print(restaurants)

# go through the resturants
#for r in restaurants['businesses']: 
    #print(r)


{'businesses': [{'id': 'ATj-8RmOJln9UnCEuraEzw', 'alias': 'noodle-in-a-haystack-san-francisco', 'name': 'Noodle in a Haystack', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/BoEM-6j0592nJhFGHMu-jA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/noodle-in-a-haystack-san-francisco?adjust_creative=-v9YsR1-ec4Yo298coLw8w&amp;utm_campaign=yelp_api_v3&amp;utm_medium=api_v3_business_search&amp;utm_source=-v9YsR1-ec4Yo298coLw8w', 'review_count': 81, 'categories': [{'alias': 'ramen', 'title': 'Ramen'}, {'alias': 'newamerican', 'title': 'New American'}], 'rating': 4.8, 'coordinates': {'latitude': 37.78066412799114, 'longitude': -122.46872181182351}, 'transactions': [], 'price': '$$$$', 'location': {'address1': '4601 Geary Blvd', 'address2': '', 'address3': '', 'city': 'San Francisco', 'zip_code': '94118', 'country': 'US', 'state': 'CA', 'display_address': ['4601 Geary Blvd', 'San Francisco, CA 94118']}, 'phone': '', 'display_phone': '', 'distance': 3586.2280585938006}, {'id'

In [61]:
# go through the restaurants and work out what needs to be stored in SQL
for r in restaurants['businesses']:
    category_titles = [category['title'] for category in r.get('categories')]  # collects all category titles
    
    restaurant_info = {'id': r.get('id'),
                      'name': r.get('name'),
                      'address1': r.get('location').get('address1'),
                      'address2': r.get('location').get('address2'),
                      'city': r.get('location').get('city'),
                      'zip_code': r.get('location').get('zip_code'),
                      'review_count': r.get('review_count'),
                      'categories': category_titles,  # category titles, comma sep
                      'rating': r.get('rating'),
                      'price': r.get('price'),
                      'transactions': r.get('transactions'),
                      'latitude': r.get('coordinates').get('latitude'),
                      'longitude': r.get('coordinates').get('longitude')
                    }
    print(restaurant_info)


{'id': 'ATj-8RmOJln9UnCEuraEzw', 'name': 'Noodle in a Haystack', 'address1': '4601 Geary Blvd', 'address2': '', 'city': 'San Francisco', 'zip_code': '94118', 'review_count': 81, 'categories': ['Ramen', 'New American'], 'rating': 4.8, 'price': '$$$$', 'transactions': [], 'latitude': 37.78066412799114, 'longitude': -122.46872181182351}
{'id': 'kvFI8S8XuE0793DBR0wSbg', 'name': 'San Ho Won', 'address1': '2170 Bryant St', 'address2': '', 'city': 'San Francisco', 'zip_code': '94110', 'review_count': 566, 'categories': ['Korean', 'Barbeque'], 'rating': 4.6, 'price': '$$$$', 'transactions': [], 'latitude': 37.75961276313876, 'longitude': -122.4102472}
{'id': 'KarBZT88nFrtDTd-rWkvFQ', 'name': 'Friends Only', 'address1': '1501 California St', 'address2': '', 'city': 'San Francisco', 'zip_code': '94109', 'review_count': 24, 'categories': ['Sushi Bars'], 'rating': 4.8, 'price': '$$$$', 'transactions': [], 'latitude': 37.79071643719494, 'longitude': -122.4192309879741}
{'id': 'cCVR9W_L2rqexfKAjfBTz

In [57]:
# Function to create a SQL table
def create_sql_table(db_name, table_name, table_info):
    try:
        # Connect to server
        conn = mysql.connector.connect(host='localhost', user='root', password='SaZhu0809')
        cursor = conn.cursor()
        
        # Create database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        
        # Create table if it doesn't exist
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {db_name}.{table_name} {table_info}")
        
    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        cursor.close()
        conn.close()

# Function to insert data into the table
def insert_data(db_name, table_name, data):
    try:
        # Connect to the MySQL database
        conn = mysql.connector.connect(host='localhost', user='root', password='SaZhu0809', database=db_name)
        cursor = conn.cursor()

        # Create a parameterized SQL query
        placeholders = ', '.join(['%s'] * len(data))
        columns = ', '.join(data.keys())
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Execute the query with the values from the data dictionary
        cursor.execute(sql, list(data.values()))

        # Commit the transaction
        conn.commit()

    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()



In [62]:
# My database and table name
SQL_DB = "bax422_final_price4"
SQL_TABLE_NAME = "Restaurant_with_Price4"

# SQL table structure
SQL_TABLE_USER_INFO = """
(
    id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(100),
    address1 VARCHAR(100),
    address2 VARCHAR(100),
    city VARCHAR(100),
    zip_code INT,
    review_count INT,
    categories VARCHAR(100),
    rating FLOAT,
    price VARCHAR(10),
    transactions VARCHAR(100),
    latitude FLOAT,
    longitude FLOAT
)
"""

# Create your database and table
create_sql_table(SQL_DB, SQL_TABLE_NAME, SQL_TABLE_USER_INFO)


In [63]:
# Go through and store the data in SQL database 
for r in restaurants['businesses']:
    # Collect all category titles
    category_titles = ', '.join(category['title'] for category in r['categories'])

    # Prepare the restaurant info dictionary
    restaurant_info = {
        'id': r['id'],
        'name': r['name'],
        'address1': r['location']['address1'],
        'address2': r['location'].get('address2', ''),  # Using get to avoid KeyError if 'address2' does not exist
        'city': r['location']['city'],
        'zip_code': r['location']['zip_code'],
        'review_count': r['review_count'],
        'categories': category_titles,  # Categories as a comma-separated string
        'rating': r['rating'],
        'price': r['price'],
        'transactions': ', '.join(r['transactions']),  # transactions is a list
        'latitude': r['coordinates']['latitude'],
        'longitude': r['coordinates']['longitude']
    }

    # Insert data into SQL table
    insert_data(SQL_DB, SQL_TABLE_NAME, restaurant_info)

: 